In [9]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('../dataset/train/train.csv')

data = dataset.loc[:, ['id', 'sentence', 'subject_entity', 'object_entity', 'source']]
label = dataset['label']

print(dataset.shape)

(32423, 6)


In [10]:
print(label.value_counts())

no_relation                            9515
org:top_members/employees              4279
per:employee_of                        3567
per:title                              2098
org:member_of                          1864
org:alternate_names                    1318
per:origin                             1234
org:place_of_headquarters              1194
per:date_of_birth                      1130
per:alternate_names                     998
per:spouse                              795
per:colleagues                          534
per:parents                             518
org:founded                             450
org:members                             420
per:date_of_death                       417
org:product                             380
per:children                            304
per:place_of_residence                  192
per:other_family                        190
per:place_of_birth                      166
org:founded_by                          155
per:product                     

In [11]:
replace_label = ['no_relation', 'per:alternate_names', 'org:alternate_names', 'per:other_family', 'per:colleagues', 'per:siblings', 'per:spouse']

newData = []
id = len(dataset)

for i in range(len(dataset)):
    if dataset.iloc[i, 4] in replace_label:
        data = dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        newData.append(data)
        id += 1

<ipython-input-11-c69a7c88e947>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['id'] = id
<ipython-input-11-c69a7c88e947>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['subject_entity'] = data['object_entity']
<ipython-input-11-c69a7c88e947>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_entity'] = tmp


In [ ]:
replace_label = ['no_relation', 'per:alternate_names', 'org:alternate_names', 'per:other_family', 'per:colleagues', 'per:siblings', 'per:spouse']

newData = []
id = len(dataset)

for i in range(len(dataset)):
    if dataset.iloc[i, 4] in replace_label:
        data = dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        newData.append(data)
        id += 1

In [34]:
newDF = pd.DataFrame(newData, columns=['id', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'])
newDF.to_csv('../dataset/train/addDataset.csv',index=False)
#dataset = dataset.append(newDF, ignore_index=True)

#print(dataset.shape)

#dataset.to_csv('../dataset/train/newTrain.csv',index=False)

(36446, 6)


In [35]:
print(dataset['label'].value_counts())

no_relation                            9534
org:top_members/employees              4284
per:employee_of                        3573
org:alternate_names                    2640
per:title                              2103
per:alternate_names                    2002
org:member_of                          1866
per:spouse                             1590
per:origin                             1234
org:place_of_headquarters              1195
per:date_of_birth                      1130
per:colleagues                         1068
per:parents                             520
org:founded                             450
org:members                             420
per:date_of_death                       418
org:product                             380
per:other_family                        380
per:children                            304
per:siblings                            272
per:place_of_residence                  193
per:place_of_birth                      166
org:founded_by                  

In [36]:
"""for i in range(len(dataset)):
    if dataset.iloc[i,4] == 'org:members':
        print(dataset.iloc[i,:])"""

"for i in range(len(dataset)):\n    if dataset.iloc[i,4] == 'org:members':\n        print(dataset.iloc[i,:])"